<a href="https://colab.research.google.com/github/FuadKhalit/Pyspark_Repo/blob/main/Car_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install the Dependencies**


In [12]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
#!tar xf spark-2.4.7-bin-hadoop2.7.tgz
#!pip install -q findspark

# **Import Library**

In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
import findspark
findspark.init()
from google.colab import files
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder

In [14]:
# start spark session locally
sc = SparkSession.builder.master("local[*]").getOrCreate()

# **Upload Dataset**

In [15]:
#upload file from google drive
file_path = '/content/drive/MyDrive/CarFeatureMSRP.csv'
car = sc.read.format('csv').option('header', True).option('inferSchema', True).load(file_path)
car.show()

#Upload file directly from Local Machine
#files.upload()
#car = sc.read.csv('CarFeatureMSRP.csv',inferSchema=True, header=True)

+----+----------+----+--------------------+---------+----------------+-----------------+-----------------+---------------+--------------------+------------+-------------+-----------+--------+----------+-----+
|Make|     Model|Year|    Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|    Driven_Wheels|Number of Doors|     Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity| MSRP|
+----+----------+----+--------------------+---------+----------------+-----------------+-----------------+---------------+--------------------+------------+-------------+-----------+--------+----------+-----+
| BMW|1 Series M|2011|premium unleaded ...|      335|               6|           MANUAL| rear wheel drive|              2|Factory Tuner,Lux...|     Compact|        Coupe|         26|      19|      3916|46135|
| BMW|  1 Series|2011|premium unleaded ...|      300|               6|           MANUAL| rear wheel drive|              2|  Luxury,Performance|     Compact|  Conver

# **Data Exploration**

In [16]:
car.printSchema()
car.describe().toPandas().transpose()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Engine Fuel Type: string (nullable = true)
 |-- Engine HP: integer (nullable = true)
 |-- Engine Cylinders: integer (nullable = true)
 |-- Transmission Type: string (nullable = true)
 |-- Driven_Wheels: string (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Vehicle Style: string (nullable = true)
 |-- highway MPG: integer (nullable = true)
 |-- city mpg: integer (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- MSRP: integer (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
Make,11914,None,None,Acura,Volvo
Model,11914,745.5822222222222,1490.8280590623795,1 Series,xD
Year,11914,2010.384337753903,7.5797398875957995,1990,2017
Engine Fuel Type,11911,None,None,diesel,regular unleaded
Engine HP,11845,249.38607007176023,109.19187025917194,55,1001
Engine Cylinders,11884,5.628828677213059,1.78055934824622,0,16
Transmission Type,11914,None,None,AUTOMATED_MANUAL,UNKNOWN
Driven_Wheels,11914,None,None,all wheel drive,rear wheel drive
Number of Doors,11908,3.4360933825999327,0.8813153865835529,2,4


In [17]:
def replace(column, value):
  return when(column != value, column).otherwise(lit(None))
car = car.withColumn("Market Category", replace (col("Market Category"), "N/A"))

In [18]:
#show missing value in dataset
car.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in car.columns]).show()

+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|Make|Model|Year|Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|Driven_Wheels|Number of Doors|Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity|MSRP|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|   0|    0|   0|               3|       69|              30|                0|            0|              6|           3742|           0|            0|          0|       0|         0|   0|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+



In [19]:
car = car.drop("Market Category")
car = car.na.drop()
print((car.count(), len(car.columns)))

(11812, 15)


This will print the number of row (11,812) & column (12) after dropping

# **Random Forest Pipeline**

In [20]:
assembler = VectorAssembler(inputCols=['Year', 'Engine HP', 'Engine Cylinders', 
                                       'Number of Doors', 'highway MPG', 'city mpg', 'Popularity', 
                                       ], outputCol='Attributes') #combine the value of input into single vector & store to attributes

regressor = RandomForestRegressor(featuresCol='Attributes', labelCol='MSRP')

pipeline = Pipeline(stages=[assembler, regressor])

pipeline.write().overwrite().save("pipeline") #pipeline is save in folder name 'pipeline'

In [21]:
!ls #check the folder in filesystem

drive	     spark-2.4.7-bin-hadoop2.7	      spark-2.4.7-bin-hadoop2.7.tgz.2
pipeline     spark-2.4.7-bin-hadoop2.7.tgz
sample_data  spark-2.4.7-bin-hadoop2.7.tgz.1
